In [26]:
import tkinter  # 아래와 같음.
from tkinter import *  # tkinter를 이용하기 위한 라이브러리 포함
import threading  # 멀티 쓰레드를 이용하기 위해서 포함시킨 라이브러리
import cv2  # 오픈CV 라이브러리 포함
from PIL import ImageGrab
import numpy as np
import pytesseract  #파이테서렉트 포함시키기
from PIL import Image #PIL 라이브러리 포함시키기

def mouse_handler(event, x, y, flags, param):
    global drawing
    global a, b, c, d
    if event == cv2.EVENT_LBUTTONDOWN:  # 마우스를 누른 상태
        drawing = True
        a, b = x, y

    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 이동
        if drawing == True:  # 마우스를 누른 상태 일경우
            c, d = x, y


    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False  # 마우스를 때면 상태 변경
        c, d = x, y


a = -1
b = -1
c = -1
d = -1
drawing = False
toy = ' '

class connect(threading.Thread):
    def __init__(self):  # threading.Thread를 사용하기 위해 선언함
        threading.Thread.__init__(self)  # initialize가 시작한다는 의미고 위에서 선언한 threading.Thread를 시작함.

    def run(self):  # 런을 이용하면 아래에서 self.gui()를 위에 써주지 않고도 정의와 동시에 동작시킨다.
        while (True):
            screen = np.array(
                ImageGrab.grab())  # np-넘파이 >> 넘파이 배열에 이미지그랩을 이용하여 grab()이므로 화면 전체를 배열로 스크린에 저장 grab('500x300')으로 캡처범위 설정가능
            global img  # 전역변수 선언
            img = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)  # OpenCV가 인식 할 수 있도록 형식을 맞추어 스크린을 img에 저장
            cv2.setMouseCallback('image', mouse_handler)  # 위에 정의한 마우스 핸들러 함수를 실행하되 image창 내에 콜백(불러온다)하여 핸들러를 실행
            cv2.rectangle(img, (int(a * 2), int(b * 2)), (int(c * 2), int(d * 2)), (0, 255, 0),
                          10)  # 전체화면 img에 주어진 좌표 ab,cd를 이용하여 색에 맞는 크기 10인 사각형을 그린다.
            dst = cv2.resize(img, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
            cv2.imshow('image', dst)
            # cv2.imshow('image', img)       #위에서 마우스 핸들러하는 곳이 image이므로 여기서도 image로 해준다. 그리고 이미지를 보여주는 기능을 실행 아직 전체화면에서이다.
            # 'q'키를 누르면 종료합니다
            if cv2.waitKey(25) & 0xFF == ord('q'):  # 키의 입력을 기다리는 함수 입력시간 25m초를 대기한다. ord가 입력받은 키보드 q
                cv2.destroyAllWindows()  # OpenCV에 관련된 모든 창을 끌때 사용한다.
                break  # 나가는 동작이 while문안에서 계속 실행되면 안되기에 탈출

class trans(threading.Thread) :
    def __init__(self):  # threading.Thread를 사용하기 위해 선언함
        threading.Thread.__init__(self)

    def run(self):
            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\\tesseract.exe'  # 테서렉트의 저장위치 선정  중요 주소앞에 r을 꼭 넣자
            global  toy
            move = pytesseract.image_to_string(r'C:\Users\user\Desktop\naver\image.png')  # 스캔할 이미지의 파일위치 선정  주소 앞에 r을 꼭 넣자
            print(move)
            toy = move

            # 중간연결파트
            f = open(r'C:\Users\user\Desktop\naver\papa.txt', 'w')
            f.write(move)
            f.close()

            # 명근파트
            with open(r'C:\Users\user\Desktop\naver\papa.txt', 'rt', encoding='UTF8') as f:
                text = f.read()

            import os
            import sys
            import urllib.request

            client_id = "yME6U804aomCqGtZLKAq"  # 개발자센터에서 발급받은 Client ID 값
            client_secret = "94ZuQml6Dg"  # 개발자센터에서 발급받은 Client Secret 값
            encText = urllib.parse.quote(text)
            data = "source=en&target=ko&text=" + encText
            url = "https://openapi.naver.com/v1/papago/n2mt"
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id", client_id)
            request.add_header("X-Naver-Client-Secret", client_secret)
            response = urllib.request.urlopen(request, data=data.encode("utf-8"))
            rescode = response.getcode()
            if (rescode == 200):
                response_body = response.read().decode('utf-8')
                # response_body 변수는 시스템 정보가 포함된 번역문이다.
                # 따라서 번역된 문장만 출력시키는 작업을 하였다
                posnum = response_body.find('translatedText')
                only_result_text = response_body[posnum + 17:-4]
                print(only_result_text)
                with open(r'C:\Users\user\Desktop\naver\papa.txt', 'w', encoding='UTF8') as f:
                    f.write(only_result_text)
            else:
                print("Error Code:" + rescode)

class GUIStart(tkinter.Tk):
    def __init__(self):
        tkinter.Tk.__init__(self)  # GUI를 사용하기 위해 선언함
        self.gui()  # 맨처음 실행되는 함수안에 다음 함수 실행하기 위해 쓴 코드

    def gui(self):
        self.title("Server")
        self.geometry("500x350")
        self.resizable(False, False)
        self.config(background="white")

        #문자를 출력하던부분
        # global str
        # str = '안녕\n 반가워 곰돌아' + toy
        # text = tkinter.Text(self)
        # text.insert(tkinter.CURRENT, "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    ")
        # text.insert(18.3, str)
        # text.pack()

        la_ser = Label(self, text="Server", font=("바탕", 30), foreground="blue", background="white")
        la_ser.place(x=125, y=20)

        btn_sock = Button(self, text="Start", background="white", command=self.startprocessing)
        btn_sock.place(x=50, y=150, width=100, height=50)

        btn_sock = Button(self, text="Capture", background="white", command=self.capture)  # command를 이용해 명령한다.
        btn_sock.place(x=200, y=150, width=100, height=50)

        btn_sock = Button(self, text="Trans", background="white", command=self.Trans)
        btn_sock.place(x=350, y=150, width=100, height=50)


    def startprocessing(self):
        connect().start()

    def capture(self):
        # cv2.imwrite('image.png', img[b:d, a:c])   #원래 방식
        cv2.imwrite(r'C:\Users\user\Desktop\naver\image.png', img[b * 2:d * 2, a * 2:c * 2])  # 원하는 위치에 저장(바탕화면)

    def Trans(self):
        trans().start()

if __name__ == '__main__':  # 메인함수, 멀티쓰레드를 동작시킨다.
    root = GUIStart()
    root.mainloop()